# Building event features

> These are Go notebooks: In order to use the GoNB Jupyter Kernel, please install GoNB from here: https://github.com/janpfeifer/gonb

Note also that for local package development, you can put: `!*go mod edit -replace "github.com/umbralcalc/trywizard=/path/to/trywizard"` at the top of any cell.

In this notebook we're building event features from local csv files which aggregate information over all the data sources listed in the `README.md`.

In [ ]:
!*go mod edit -replace "github.com/umbralcalc/trywizard=/home/robert/Code/trywizard"

In [ ]:
import (
	"os"
	"github.com/umbralcalc/trywizard/pkg/features"
	gonb_echarts "github.com/janpfeifer/gonb-echarts"
)

%%

df := features.GetRawEventsDataFrame()
cdf := features.TransformToEventCounts(&df)
scatter := features.NewDataFrameScatter(
	&cdf, 
	"minute", 
	"event_type_COUNT",
	"event_type",
)

gonb_echarts.Display(scatter, "width: 1024px; height:400px; background: white;")
